In [2]:
# Check the gene names
import pandas as pd
import requests

# Read Document 1 (assumes an Excel file with a column "Protein" containing gene names)
df1 = pd.read_excel('/Users/conny/Desktop/AlphaFold/Paper 2 data.xlsx', header=11)

# Now you can access the 'Protein' column
protein_column = df1['T: Protein IDs'].dropna().unique()
print(protein_column)
#gene_names = df1['Protein'].dropna().unique()  # unique gene names, drop NaNs if any


['Q8TDN6' 'Q6PK04;I3L0U5;I3L4F6'
 'P46087;P46087-4;P46087-2;P46087-3;F5GYR3;F5GWB7;F5H709;F5H5X6;F5H359;F5H8G6'
 'Q92522' 'Q9NVP1;H7C452'
 'O76021;J3QSV6;I3L3U9;B4DJ58;I3L3C4;I3L234;I3L1A3;J3KPU7;I3L2F2;I3L4T8;I3L1Y4'
 'Q9NQ55;Q9NQ55-2;C9J3F9;A8MV53;C9J3W3;C9JHF7' 'Q9NR30;Q9NR30-2'
 'O00567;Q5JXT2;H0YDU4;H0Y653' 'Q9BYG3;C9J808;C9J6C5;E9PFS2;H7BZL0'
 'Q9Y5J1;J3KSR7;J3QR85;J3QRG0;F6VJE8;J3QLD6' 'Q9UKD2'
 'Q13838;Q13838-2;F8VQ10;Q5STU3;H0Y7G4;B4DP52;F6WLT2;F8VZL8;H0Y7X6;F6S4E6;F6UN21;F6TRA5;H0Y800;H0Y557;F6UJC5;E9PP12;H0Y6N8;H0Y4V8;H0Y6G9;F8W0J0;H0Y400;B4DIJ6;F6V772;F6QYI9;F6R6M7;K7ENP6;F6QWI5;K7EL56;K7EIL8;F6U6E2;K7EN69;F6S2B7;F6SXL5;E7EMX2;K7EN25;H0YCC6;H0Y6J9;H0Y6Y9'
 'Q9BQG0;Q9BQG0-2;I3L1L3;I3L2H8;I3L311' 'Q15050'
 'P22087;M0QXL5;M0R0P1;M0R299;M0R2Q4;M0R2U2;M0R1H0;M0R2B0;M0QXC9'
 'Q9UKM9;Q9UKM9-2;Q5QPL9;Q5QPM1;Q5QPM0;Q5QPM2' 'Q9H7B2;Q5VXN0;Q5VXM9'
 'P38159;P38159-2;H3BT71;H0Y6E7;B3KRG5;H3BUY5;H3BR27;O75526;H3BNC1;Q8IZ40;Q8N7X1'
 'O43390;O43390-2;Q2L7G6;E7ETM7'
 'Q5T3I0;Q5T3I0-3;E9PAV9

In [4]:
import requests
import sys
import json
import pandas as pd
from tqdm import tqdm  # <-- Add this import

# Extract the Protein ID from the column
protein_ids = protein_column

# API Parameters
params = {"fields": ["sequence"]}
headers = {"accept": "application/json"}

# Output file
output_file = "p2_protein_sequences.fasta"

def fetch_sequence(protein_id):
    url = f"https://rest.uniprot.org/uniprotkb/{protein_id}"
    response = requests.get(url, headers=headers, params=params)
    if response.ok:
        data = response.json()
        sequence_info = data.get("sequence", {})
        return sequence_info.get("value", None)
    else:
        print(f"Warning: Failed to fetch {protein_id}", file=sys.stderr)
        return None

results = {}
# Wrap tqdm around the outer loop
for entry in tqdm(protein_ids, desc="Fetching sequences"):
    protein_ids_split = entry.split(";")
    for protein_id in protein_ids_split:
        protein_id = protein_id.strip()
        if protein_id and protein_id not in results:
            sequence = fetch_sequence(protein_id)
            if sequence:
                results[protein_id] = sequence
            else:
                results[protein_id] = "Sequence Not Found"

with open(output_file, "w") as file:
    for protein_id, sequence in results.items():
        file.write(f">{protein_id}\n{sequence}\n")

print(f"Protein sequences saved to {output_file}")


Fetching sequences: 100%|██████████| 469/469 [17:05<00:00,  2.19s/it]

Protein sequences saved to p2_protein_sequences.fasta
